<a href="https://colab.research.google.com/github/minnnnji/Machine_Learning/blob/master/logistic_regressor_cabbage_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"minzzzzi","key":"74a980fb52033cf0ead89c84120a4a9c"}'}

In [3]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle

Uninstalling kaggle-1.5.8:
  Successfully uninstalled kaggle-1.5.8
     |████████████████████████████████| 1.5MB 2.5MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 59 kB 1.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73274 sha256=4fa4c1a7f5b47f0ff4dc85fdbf7a7a413c5e7263ff2ba1eb7f790563edbecca1
  Stored in directory: /root/.cache/pip/wheels/cf/aa/f0/ed1179bbcd729b29d0dfda59826fb3b55f0a4a0c3f713c1c82
Successfully built kaggle


In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c mlregression-cabbage-price
!unzip mlregression-cabbage-price.zip

  0% 0.00/29.8k [00:00<?, ?B/s]
100% 29.8k/29.8k [00:00<00:00, 20.9MB/s]
Archive:  mlregression-cabbage-price.zip
  inflating: sample_submit.csv       
  inflating: test_cabbage_price.csv  
  inflating: train_cabbage_price.csv  


In [6]:
import torch
import numpy as np
import pandas as pd
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
device = torch.device("cuda")
sc = StandardScaler()
torch.manual_seed(1)

In [23]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()

In [7]:
train = pd.read_csv('train_cabbage_price.csv')
test = pd.read_csv('test_cabbage_price.csv')
submit = pd.read_csv('sample_submit.csv')

In [8]:
train[:5]

,year,avgTemp,minTemp,maxTemp,rainFall,avgPrice
0,20100101,-4.9,-11.0,0.9,0.0,2123
1,20100102,-3.1,-5.5,5.5,0.8,2123
2,20100103,-2.9,-6.9,1.4,0.0,2123
3,20100104,-1.8,-5.1,2.2,5.9,2020
4,20100105,-5.2,-8.7,-1.8,0.7,2060


In [38]:
train = np.array(train)
x_train = train[:,:-1]
y_train = train[:,-1]
#x_train = torch.FloatTensor(train[:,:-1])
#y_train = torch.FloatTensor(train[:,-1])

# Test data 
test = np.array(test)
#x_test = torch.FloatTensor(x_test)
for i in range(len(x_train)):
  x_train[i][0]%=10000.0
  x_train[i][0]/=100.0
for i in range(len(test)):
  test[i][0]%=10000.0
  test[i][0]/=100.0

x_train[:5]

array([[ 1.01e-06, -4.90e+00, -1.10e+01,  9.00e-01,  0.00e+00],
       [ 1.02e-06, -3.10e+00, -5.50e+00,  5.50e+00,  8.00e-01],
       [ 1.03e-06, -2.90e+00, -6.90e+00,  1.40e+00,  0.00e+00],
       [ 1.04e-06, -1.80e+00, -5.10e+00,  2.20e+00,  5.90e+00],
       [ 1.05e-06, -5.20e+00, -8.70e+00, -1.80e+00,  7.00e-01]])

In [39]:
x_train_std = sc.fit_transform(x_train)
test_std = sc.fit_transform(test)


In [26]:
xx_train_std = mm.fit_transform(x_train)
xx_train_std[:5]

array([[0.        , 0.13316583, 0.0942928 , 0.16097561, 0.        ],
       [0.00088496, 0.17839196, 0.23076923, 0.27317073, 0.00837696],
       [0.00176991, 0.18341709, 0.19602978, 0.17317073, 0.        ],
       [0.00265487, 0.21105528, 0.24069479, 0.19268293, 0.0617801 ],
       [0.00353982, 0.12562814, 0.15136476, 0.09512195, 0.00732984]])

In [52]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(C=700, random_state = 11, solver='liblinear',max_iter = 1000)
l = logit.fit(xx_train_std,y_train)
train_pred = logit.predict(xx_train_std)
test_pred = logit.predict(test)

In [53]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train,train_pred))

0.07485166590597901


In [ ]:
test_pred.reshape(-1,1)


In [55]:
for i in range(len(test)):
   submit['Expected'][i] = test_pred[i];

submit[:5]

,Id,Expected
0,0,2507
1,1,4521
2,2,4521
3,3,4521
4,4,2507


In [56]:
submit.to_csv('submit.csv', mode='w', header=True, index= False)

In [57]:
!kaggle competitions submit -c mlregression-cabbage-price -f submit.csv -m "Message"

100% 6.33k/6.33k [00:08<00:00, 757B/s]
Successfully submitted to [ML]Regression : Cabbage price